In [ ]:
import sys

# code for enabling this notebook to work within cursor
coralme_dir = '/home/chris/zuniga/coralme/'
sys.path.insert(0, coralme_dir)

import importlib
import coralme
import coralme.solver.solver
import coralme.builder.main
import coralme.core.model
import cobra
from coralme.util.helpers import get_nlp, optimize
from contextlib import contextmanager
import os
import pandas as pd
import pickle

@contextmanager
def suppress_output():
    # Save file descriptors
    stdout_fd = os.dup(1)
    stderr_fd = os.dup(2)
    
    # Open null device
    null_fd = os.open(os.devnull, os.O_RDWR)
    
    # Redirect stdout and stderr to null device
    os.dup2(null_fd, 1)
    os.dup2(null_fd, 2)
    
    try:
        yield
    finally:
        # Restore file descriptors
        os.dup2(stdout_fd, 1)
        os.dup2(stderr_fd, 2)
        
        # Clean up
        os.close(null_fd)
        os.close(stdout_fd)
        os.close(stderr_fd)

# load biolog data
#biolog_data = pd.read_excel('../species_files/Zymomonas_mobilis/verification_data/biolog_10_1016_j_synbio_2023_07_001.xlsx')
path = os.path.join(coralme_dir, 'species_files', 'Zymomonas_mobilis', 'verification_data', 'biolog_semi_processed.csv')
biolog_data = pd.read_csv(path, index_col = 0)

: 

In [2]:
# run through all models, saving off growth capabilities
base_dir = os.path.join(coralme_dir, 'species_files', 'Pseudomonas_files')
for f in os.listdir(os.path.join(base_dir, 'individual_species')):
    # load in ME_model
    out_path = os.path.join(base_dir, 'individual_species', f, 'outputs')
    if os.path.exists(os.path.join(out_path, 'growth_check_done.pkl')):
        print(str(f)+' : solution already run, skipping')
        continue
    out_dict = {}
    opts = os.listdir(out_path)
    model_name = None
    for opt in opts:
        if 'step3' in opt:
            model_name = opt
    if not model_name:
        print(str(f)+': model doesn\'t exist')
    print(str(f)+' : working...', end = '')
    ME_model = coralme.io.pickle.load_pickle_me_model(os.path.join(out_path, model_name))
    

    # find the exchange reactions
    carbon_exchanges = []
    nitrogen_exchanges = []
    sulfur_exchanges = []
    phosphorus_exchanges = []
    for rxn in ME_model.reactions:
        if 'EX_' in rxn.id:
            elements = set()
            for metab in list(rxn.metabolites.keys()):
                for ele in metab.elements.keys():
                    elements.add(ele)
            if 'C' in elements:
                carbon_exchanges.append(rxn)
            if 'N' in elements:
                nitrogen_exchanges.append(rxn)
            if 'S' in elements:
                sulfur_exchanges.append(rxn)
            if 'P' in elements:
                phosphorus_exchanges.append(rxn)
    
    # run for each set of exchange categories
    for cat, cat_list in zip(['carbon', 'nitrogen', 'sulfur', 'phosphorus'], [carbon_exchanges, nitrogen_exchanges, sulfur_exchanges, phosphorus_exchanges]):
        # dictionary update, skip if already run
        if cat not in out_dict:
            out_dict.update({cat : {}})

        # first turn on all other exchanges
        for rxn in carbon_exchanges+nitrogen_exchanges+sulfur_exchanges+phosphorus_exchanges:
            if rxn not in cat_list:
                rxn.lower_bound = -1000
                rxn.upper_bound = 1000
        # loop through each carbon exchange, turn on one at a time
        for rxn in cat_list:
            if rxn.id in out_dict[cat]:
                continue

            # turn off all other carbon exchanges
            for rxn1 in cat_list:
                rxn1.lower_bound = 0
                rxn1.upper_bound = 0
            # turn on the current carbon exchange
            rxn.lower_bound = -1000
            rxn.upper_bound = 1000
            
            # run the model and save the results
            ME_model.optimize(tolerance = 1e-3)
            try:
                ME_sol = ME_model.solution
            except:
                ME_sol = None
            if ME_sol:
                out_dict[cat].update({rxn.id : ME_sol.objective_value})
            else:
                out_dict[cat].update({rxn.id : 0})
    
            # save the dictionary output
            pickle_out = open(os.path.join(out_path, 'growth_check_output.pkl'), 'wb')
            pickle.dump(out_dict, pickle_out)
            pickle_out.close()

    # save off the done check
    out_path = os.path.join(base_dir, 'individual_species', f, 'outputs')
    pickle_out = open(os.path.join(out_path, 'growth_check_done.pkl'), 'wb')
    pickle.dump(True, pickle_out)
    pickle_out.close()

CP008896 : working...Iteration	 Solution to check	Solver Status
---------	------------------	-------------
        1	1.4050280687025918	Not feasible
        2	0.7025140343512959	Not feasible
        3	0.3512570171756479	Not feasible
        4	0.1756285085878240	Not feasible
        5	0.0878142542939120	Not feasible
        6	0.0439071271469560	Not feasible
        7	0.0219535635734780	Not feasible
        8	0.0109767817867390	Not feasible
        9	0.0054883908933695	Not feasible
       10	0.0027441954466847	Not feasible
       11	0.0013720977233424	Not feasible
       12	0.0006860488616712	Not feasible
Iteration	 Solution to check	Solver Status
---------	------------------	-------------
        1	1.4050280687025918	Not feasible
        2	0.7025140343512959	Not feasible
        3	0.3512570171756479	Not feasible
        4	0.1756285085878240	Not feasible
        5	0.0878142542939120	Not feasible
        6	0.0439071271469560	Not feasible
        7	0.0219535635734780	Not feasible
        8

: 

: 

# below is the code for running one at a time in Zymomonas

In [ ]:
# load the model and create the nlp and dictionaries for solvers
me = coralme.io.pickle.load_pickle_me_model('../species_files/Zymomonas_mobilis/outputs/MEModel-step3-zymomonas-TS.pkl')
nlp = get_nlp(me)
rxn_index_dct = {r.id : me.reactions.index(r) for r in me.reactions}
met_index_dct = {m.id : me.metabolites.index(m) for m in me.metabolites}
default_ups = {
    exch : nlp.xu[rxn_index_dct[exch]] for exch in rxn_index_dct.keys()
}
default_lows = {
    exch : nlp.xl[rxn_index_dct[exch]] for exch in rxn_index_dct.keys()

In [2]:
# finding biolog substrates within data
name_to_id = {val.name : val.id for val in me.metabolites}
id_short_to_ids = {}
for name in name_to_id.values():
    short = name.split('_')[0]
    if short not in id_short_to_ids:
        id_short_to_ids.update({short : []})
    id_short_to_ids[short].append(name)
all_metabs = list(set([val.name for val in me.metabolites]))
all_ids = list(set([val.id for val in me.metabolites]))
for metab in all_metabs+all_ids:
    if 'lactic' in metab.lower():
        print(metab, end = ' ... ')
        print(name_to_id[metab])


# this takes way too long and I'm still not sure on the purpose, I don't think this is a good use of my time
# let's try automating it
new_vals = []
for index, row in biolog_data.iterrows():
    short = row['ID in iZM4_478'].split('_')[0]
    if short in id_short_to_ids:
        if len(id_short_to_ids[short]) == 1:
            new_vals.append(id_short_to_ids[short][0])
        else:
            new_vals.append(id_short_to_ids[short])
    else:
        new_vals.append('-')
biolog_data['best_coralme_guess'] = new_vals
biolog_data.to_csv('../species_files/Zymomonas_mobilis/verification_data/biolog_semi_processed.csv')

# now I need to map to the exchange reactions
metab_to_exchange = {}
for index, row in biolog_data.iterrows():
    metab = row['ID in coralme']
    exch = None
    if metab == '-': continue
    for rxn in me.metabolites.get_by_id(metab).reactions:
        if 'EX_' in rxn.id:
            exch = rxn.id

    if not exch:
        try:
            metab2 = metab.replace('_e', '_p')
            for rxn in me.metabolites.get_by_id(metab2).reactions:
                if 'EX_' in rxn.id:
                    exch = rxn.id
        except:
            pass
    if not exch:
        try:
            metab2 = metab.replace('_p', '_e')
            for rxn in me.metabolites.get_by_id(metab2).reactions:
                if 'EX_' in rxn.id:
                    exch = rxn.id
        except:
            pass

    if exch:
        metab_to_exchange.update({metab : exch})

# define the exchanges reactions per plate
carbon_exchanges = []
nitrogen_exchanges = []
sulfur_exchanges = []
phosphorus_exchanges = []
other_exchanges = []
for rxn in me.reactions:
    if 'EX_' not in rxn.id: continue
    elements = set()
    for metab in list(rxn.metabolites.keys()):
        for ele in metab.elements.keys():
            elements.add(ele)
    if 'C' in elements:
        carbon_exchanges.append(rxn.id)
    if 'N' in elements:
        nitrogen_exchanges.append(rxn.id)
    if 'S' in elements:
        sulfur_exchanges.append(rxn.id)
    if 'P' in elements:
        phosphorus_exchanges.append(rxn.id)
    else:
        other_exchanges.append(rxn.id)
plate_to_exchanges = {
    'PM1' : carbon_exchanges,
    'PM2' : carbon_exchanges,
    'PM3' : nitrogen_exchanges,
    'PM4_sulfur' : sulfur_exchanges,
    'PM4_phosphorus' : phosphorus_exchanges,
}
all_exchanges = set(carbon_exchanges + nitrogen_exchanges + sulfur_exchanges + phosphorus_exchanges + other_exchanges)

In [4]:
# loop through all exchanges, shut off one at a time to figure out what can be cut
# PM1 and PM2 were set as the only carbon source
# PM3, PM4 A1-E12, or PM4 F1–H12 were set as the only nitrogen, sulfur, or phosphorus source, respectively.

i = 0
for index, row in biolog_data.iterrows():
    i += 1
    metab = row['ID in coralme']
    if metab not in metab_to_exchange:
        continue
    interest_exc = metab_to_exchange[metab]
    plate_type = row['PM_ID'][0:3]
    if plate_type == 'PM4':
        if row['PM_ID'][5] in 'ABCDE':
            plate_type = 'PM4_sulfur'
        else:
            plate_type = 'PM4_phosphorus'
    exchanges = plate_to_exchanges[plate_type]
    
    # reset everything to defaults
    for temp_exch in rxn_index_dct.keys():
        # things that are zero lower limit with high upper limit pad a bit for nlp's tolerance sake
        if False: # this ends up causing issues it as it prevents no flux
            if default_lows[temp_exch](0) == 0 and default_ups[temp_exch](0) > 0:
                low = lambda x : 1e-4 # it's the solver tolerance
                high = default_ups[temp_exch]
                print(low(0), end = ' to ')
                print(high(0))
            elif default_ups[temp_exch](0) == 0 and default_lows[temp_exch](0) < 0:
                low = default_lows[temp_exch]
                high = lambda x : -1e-4 # it's the solver tolerance
                print(low(0), end = ' to ')
                print(high(0)) 
        low = default_lows[temp_exch]
        high = default_ups[temp_exch]
        nlp.xl[rxn_index_dct[temp_exch]] = low
        nlp.xu[rxn_index_dct[temp_exch]] = high

    # let's turn on all of the exchanges (super rich media)
    for temp_exch in all_exchanges:
        nlp.xl[rxn_index_dct[temp_exch]] = lambda x : -70
        nlp.xu[rxn_index_dct[temp_exch]] = lambda x : 70
    
    # turn off all of relevant exchanges
    for temp_exch in exchanges:
        nlp.xl[rxn_index_dct[temp_exch]] = lambda x : 0
        nlp.xu[rxn_index_dct[temp_exch]] = lambda x : 0
    
    # turn on plate exchange
    nlp.xl[rxn_index_dct[interest_exc]] = lambda x : -70
    nlp.xu[rxn_index_dct[interest_exc]] = lambda x : 70
    
    # now run the model
    with suppress_output():
        sol, basis = optimize(rxn_index_dct, met_index_dct, nlp, max_mu = 0.1, min_mu = 0.01,
            tolerance = 1e-4, maxIter = 1, precision = 'double', verbose = False, basis = None)

    print(str(i)+' / '+str(len(biolog_data)), end = ' : ')
    print(plate_type, end = ', ')
    if str(type(sol)) != '<class \'NoneType\'>' and sum(sol.fluxes.values()) != 0:
        # growth occured!
        print(metab+' can grow exclusively on = ', end = '')
        print(sol.objective_value)
    else:
        print(metab+' cannot grow exclusively on')

4 / 379 : PM1, succ_e cannot grow exclusively on
6 / 379 : PM1, asp__L_e cannot grow exclusively on
12 / 379 : PM1, ser__L_e cannot grow exclusively on
14 / 379 : PM1, glyc_e cannot grow exclusively on
17 / 379 : PM1, glcn_e cannot grow exclusively on
18 / 379 : PM1, glyc3p_e cannot grow exclusively on
19 / 379 : PM1, xyl__D_e cannot grow exclusively on
20 / 379 : PM1, lac__D_e cannot grow exclusively on
23 / 379 : PM1, glu__L_e cannot grow exclusively on
30 / 379 : PM1, fru_e cannot grow exclusively on
46 / 379 : PM1, sucr_e cannot grow exclusively on
48 / 379 : PM1, gln__L_e cannot grow exclusively on
74 / 379 : PM1, ser__L_e cannot grow exclusively on
170 / 379 : PM2, arg__L_e cannot grow exclusively on
177 / 379 : PM2, lys__L_e cannot grow exclusively on
191 / 379 : PM3, nh4_e can grow exclusively on = 0.055
197 / 379 : PM3, arg__L_e cannot grow exclusively on
199 / 379 : PM3, asp__L_e cannot grow exclusively on
201 / 379 : PM3, glu__L_e cannot grow exclusively on
202 / 379 : PM3, 

In [ ]:
# checking for any carbon exchanges hapenning
for rxn, flux in sol.fluxes.items():
    if flux == 0: continue
    # is it exchange?
    exchange_check = False
    sides = me.reactions.get_by_id(rxn).reaction.split('<=>')
    if len(sides) != 2:
        sides = me.reactions.get_by_id(rxn).reaction.split('-->')
    if len(sides[0].replace(' ', '')) == 0 or len(sides[1].replace(' ', '')) == 0:
        exchange_check = True
    if not exchange_check : continue
    carbon_check = False
    for metab in me.reactions.get_by_id(rxn).metabolites:
        elements = metab.elements
        if 'C' in metab.elements:
            carbon_check = True
    if carbon_check:
        print(rxn, end = ' : ')
        print(flux)

# old version

In [103]:
# loop through all exchanges, shut off one at a time to figure out what can be cut
carbon_exchanges = [
    'EX_gcald_e', 'EX_glc_e', 'EX_glcn_e', 'EX_glyc_e', 
    'EX_succ_e', 'EX_sucr_e', 'EX_fum_e', 'EX_fru_e', 
    'EX_g3pc_e', 'EX_g3pe_e', 'EX_g3pg_e', 'EX_g3pi_e', 
    'EX_g3ps_e', 'EX_gal_e', 'EX_gal__bD_e', 'EX_gdp_e', 
    'EX_gln__L_e', 'EX_glu__L_e', 'EX_glyc3p_e', 'EX_lac__D_e', 
    'EX_xyl__D_e', 'EX_glycogen'
]
all_exchanges = [val for val in rxn_index_dct.keys() if 'EX_' in val]


exchanges_use = all_exchanges
i = 0
for run in exchanges_use:
    # turn everything to defaults
    for exch in exchanges_use:
        nlp.xl[rxn_index_dct[exch]] = default_lows[exch]#lambda x : -10
        nlp.xu[rxn_index_dct[exch]] = default_ups[exch]#lambda x : 10

    # turn off run
    nlp.xl[rxn_index_dct[run]] = lambda x : 0
    nlp.xu[rxn_index_dct[run]] = lambda x : 0
    
    # now run the model
    with suppress_output():
        sol, basis = optimize(rxn_index_dct, met_index_dct, nlp, max_mu = 0.1, min_mu = 0.01,
    		tolerance = 1e-4, maxIter = 1, precision = 'double', verbose = False, basis = None)
    i += 1
    print(str(i)+' / '+str(len(exchanges_use)), end = ' : ')
    if str(type(sol)) != '<class \'NoneType\'>':
        # growth occured!
        print(run+' can grow without = ', end = '')
        print(sol.objective_value)
    else:
        print(run+' cannot grow without')

1 / 93 : EX_3dhq_e cannot grow without
2 / 93 : EX_3dhsk_e cannot grow without
3 / 93 : EX_4hbz_e cannot grow without
4 / 93 : EX_ac_e cannot grow without
5 / 93 : EX_acald_e cannot grow without


KeyboardInterrupt: 

In [116]:
# check for limiters
for rxn, flux in sol.fluxes.items():
    #if 'EX_' not in rxn: continue
    if flux == 0: continue
    if flux == nlp.xl[rxn_index_dct[rxn]](0):
        print(rxn, end = ' = ')
        print(flux)
    if flux == nlp.xu[rxn_index_dct[rxn]](0):
        print(rxn, end = ' = ')
        print(flux)